<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/equityindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utilities (assuming flat_rate_py, flat_vol_py are defined as in previous examples)
def flat_rate_py(value_or_date, day_counter_or_value=None, day_counter_if_date=None):
    if isinstance(value_or_date, ql.Date):
        eval_date = value_or_date
        rate_val = day_counter_or_value
        dc = day_counter_if_date
    else:
        eval_date = ql.Settings.instance().evaluationDate
        rate_val = value_or_date
        dc = day_counter_or_value

    if isinstance(rate_val, ql.Quote):
        quote_handle = ql.QuoteHandle(rate_val)
    elif isinstance(rate_val, float):
        quote_handle = ql.QuoteHandle(ql.SimpleQuote(rate_val))
    else:
        quote_handle = rate_val
    return ql.FlatForward(eval_date, quote_handle, dc)

class CommonVarsEquityIndex: # Renamed
    def __init__(self, add_todays_fixing=True):
        self.calendar = ql.TARGET()
        self.day_count = ql.Actual365Fixed()

        # Handles first
        self.interest_handle = ql.RelinkableYieldTermStructureHandle()
        self.dividend_handle = ql.RelinkableYieldTermStructureHandle()
        self.spot_q = ql.SimpleQuote(8700.0) # Store the quote object itself
        self.spot_handle = ql.RelinkableQuoteHandle(self.spot_q)


        self.equity_index = ql.EquityIndex("eqIndex", self.calendar, ql.EURCurrency(),
                                           self.interest_handle, self.dividend_handle,
                                           self.spot_handle)

        self.today = self.calendar.adjust(ql.Date(27, ql.January, 2023))
        # ql.Settings.instance().evaluationDate is set in TestCase.setUp

        if add_todays_fixing:
            self.equity_index.addFixing(self.today, 8690.0)

        # Link handles
        self.interest_handle.linkTo(flat_rate_py(0.03, self.day_count))
        self.dividend_handle.linkTo(flat_rate_py(0.01, self.day_count))
        # Spot is already linked via RelinkableQuoteHandle construction

# Python equivalent of the Flag utility for observability tests
class TestObserver(ql.Observer):
    def __init__(self):
        super(TestObserver, self).__init__()
        self.is_up = False
    def update(self):
        self.is_up = True
    def lower(self):
        self.is_up = False
    def isUp(self): # Match C++ name
        return self.is_up

class EquityIndexTests(unittest.TestCase):
    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # CommonVars constructor reads evaluationDate, so set it before initializing
        # self.common_vars will be initialized in each test method as needed
        # with different `add_todays_fixing`

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date
        # Clear fixings from IndexManager for the test index to avoid interference
        ql.IndexManager.instance().clearHistory("eqIndex")
        ql.IndexManager.instance().clearHistory("observableEquityIndex")


    def test_todays_fixing(self):
        print("Testing today's fixing...")
        vars = CommonVarsEquityIndex(add_todays_fixing=True)
        ql.Settings.instance().evaluationDate = vars.today
        tolerance = 1.0e-8

        historical_index = 8690.0
        todays_fixing_hist = vars.equity_index.fixing(vars.today)
        self.assertAlmostEqual(todays_fixing_hist, historical_index, delta=tolerance,
                               msg=f"Today's historical fixing: Actual {todays_fixing_hist}, Expected {historical_index}")

        spot_value = 8700.0
        todays_fixing_forecast = vars.equity_index.fixing(vars.today, True) # forecastTodaysFixing = True
        self.assertAlmostEqual(todays_fixing_forecast, spot_value, delta=tolerance,
                               msg=f"Today's forecast fixing: Actual {todays_fixing_forecast}, Expected {spot_value}")

    def test_todays_fixing_with_spot_as_proxy(self):
        print("Testing today's fixing with spot as proxy...")
        vars = CommonVarsEquityIndex(add_todays_fixing=False) # No historical fixing for today
        ql.Settings.instance().evaluationDate = vars.today
        tolerance = 1.0e-8

        spot_value = 8700.0
        # When historical not added, and forecastTodaysFixing=False (default), it should use spot
        fixing_val = vars.equity_index.fixing(vars.today)
        self.assertAlmostEqual(fixing_val, spot_value, delta=tolerance,
                               msg=f"Today's fixing (no hist): Actual {fixing_val}, Expected {spot_value}")

    def test_fixing_forecast(self):
        print("Testing fixing forecast...")
        vars = CommonVarsEquityIndex()
        ql.Settings.instance().evaluationDate = vars.today
        tolerance = 1.0e-8
        forecasted_date = ql.Date(20, ql.May, 2030)

        forecast_val = vars.equity_index.fixing(forecasted_date)

        # Expected: Spot(0) * D(q, T) / D(r, T)
        # D(q,T) = exp(-q*T) => D(q,T)/D(r,T) = exp(-(q-r)T) = exp((r-q)T)
        # Spot(0) * P(q,T) / P(r,T)
        # P(q,T) is discount factor using dividend yield q
        # P(r,T) is discount factor using risk-free rate r
        expected_forecast = vars.spot_handle.value() * \
                            vars.dividend_handle.discount(forecasted_date) / \
                            vars.interest_handle.discount(forecasted_date)

        self.assertAlmostEqual(forecast_val, expected_forecast, delta=tolerance,
                               msg=f"Fixing forecast: Actual {forecast_val}, Expected {expected_forecast}")

    def test_fixing_forecast_without_dividend(self):
        print("Testing fixing forecast without dividend...")
        vars = CommonVarsEquityIndex()
        ql.Settings.instance().evaluationDate = vars.today
        tolerance = 1.0e-8
        forecasted_date = ql.Date(20, ql.May, 2030)

        # Clone with empty dividend handle
        equity_index_ex_div = vars.equity_index.clone(vars.interest_handle,
                                                     ql.YieldTermStructureHandle(), # Empty handle
                                                     vars.spot_handle)
        # Re-add historical fixings if clone doesn't copy them
        if vars.equity_index.hasHistoricalFixings(): # Check if the original had it
            equity_index_ex_div.addFixing(vars.today, 8690.0)


        forecast_val = equity_index_ex_div.fixing(forecasted_date)
        # Expected: Spot(0) / D(r, T)
        expected_forecast = vars.spot_handle.value() / vars.interest_handle.discount(forecasted_date)

        self.assertAlmostEqual(forecast_val, expected_forecast, delta=tolerance,
                               msg=f"Fixing forecast (no div): Actual {forecast_val}, Expected {expected_forecast}")

    def test_fixing_forecast_without_spot(self):
        print("Testing fixing forecast without spot handle...")
        vars = CommonVarsEquityIndex(add_todays_fixing=True) # Ensure today's fixing exists
        ql.Settings.instance().evaluationDate = vars.today
        tolerance = 1.0e-8
        forecasted_date = ql.Date(20, ql.May, 2030)

        # Clone with empty spot handle
        equity_index_ex_spot = vars.equity_index.clone(vars.interest_handle,
                                                       vars.dividend_handle,
                                                       ql.QuoteHandle()) # Empty handle
        # Re-add historical fixings if clone doesn't copy them
        equity_index_ex_spot.addFixing(vars.today, 8690.0) # From CommonVars

        forecast_val = equity_index_ex_spot.fixing(forecasted_date)
        # Expected: PastFixing(today) * D(q, T_today_to_forecast) / D(r, T_today_to_forecast)
        # Discount factors are from 'today' (eval date) to 'forecasted_date'
        # D(q,T)/D(r,T) is effectively exp((r-q)T) using rates from 'today'
        expected_forecast = equity_index_ex_spot.pastFixing(vars.today) * \
                            vars.dividend_handle.discount(forecasted_date) / \
                            vars.interest_handle.discount(forecasted_date)

        self.assertAlmostEqual(forecast_val, expected_forecast, delta=tolerance,
                               msg=f"Fixing forecast (no spot): Actual {forecast_val}, Expected {expected_forecast}")

    def test_fixing_forecast_without_spot_and_historical_fixing(self):
        print("Testing fixing forecast without spot handle and historical fixing...")
        vars = CommonVarsEquityIndex(add_todays_fixing=False) # No fixing for today
        ql.Settings.instance().evaluationDate = vars.today
        forecasted_date = ql.Date(20, ql.May, 2030)

        equity_index_ex_spot_no_hist = vars.equity_index.clone(
            vars.interest_handle, vars.dividend_handle, ql.QuoteHandle()
        )
        # No historical fixings added to this cloned index or its original

        with self.assertRaisesRegex(RuntimeError, "Cannot forecast.*missing both spot and historical"):
            equity_index_ex_spot_no_hist.fixing(forecasted_date)

    def test_spot_change(self):
        print("Testing spot change...")
        vars = CommonVarsEquityIndex()
        ql.Settings.instance().evaluationDate = vars.today
        tolerance = 1.0e-8

        new_spot_quote_obj = ql.SimpleQuote(9000.0)
        vars.spot_handle.linkTo(new_spot_quote_obj)

        # EquityIndex.spot() should return a QuoteHandle. We need its value.
        # The C++ `vars.equityIndex->spot()->value()` gets value from Quote inside Handle.
        # Python `vars.equity_index.spot()` returns the Handle.
        # `vars.equity_index.spot().value()` gets the value from the Quote inside the Handle.
        self.assertAlmostEqual(vars.equity_index.spot().value(), new_spot_quote_obj.value(), delta=tolerance,
                               msg="Spot re-link to new value failed")

        vars.spot_handle.linkTo(vars.spot_q) # Link back to original SimpleQuote object
        self.assertAlmostEqual(vars.equity_index.spot().value(), vars.spot_q.value(), delta=tolerance,
                               msg="Spot re-link back to old value failed")

    def test_error_when_invalid_fixing_date(self):
        print("Testing error when invalid fixing date is used...")
        vars = CommonVarsEquityIndex()
        ql.Settings.instance().evaluationDate = vars.today

        # Jan 1, 2023 is a Sunday, not a TARGET holiday but maybe invalid for index if calendar implies business days
        # The error "is not valid" usually means it's not a business day in the index's calendar.
        # TARGET() includes weekends.
        # Let's check if Jan 1, 2023 is a business day in TARGET. It is.
        # The error might be related to something else, or if TARGET considers New Year's Day a holiday.
        # TARGET calendar: New Year's Day (Jan 1st), Good Friday, Easter Monday, Labour Day (May 1st), Christmas (Dec 25th), Boxing Day (Dec 26th).
        # Jan 1st, 2023 is a Sunday. The next business day is Jan 2nd.
        # The error likely means the date is not a business day *for that index's calendar*.
        invalid_date = ql.Date(1, ql.January, 2023) # Sunday
        self.assertFalse(vars.calendar.isBusinessDay(invalid_date))

        with self.assertRaisesRegex(RuntimeError, "is not valid"): # Error message might differ slightly
            vars.equity_index.fixing(invalid_date)

    def test_error_when_fixing_missing(self):
        print("Testing error when required fixing is missing...")
        vars = CommonVarsEquityIndex(add_todays_fixing=True) # Today's fixing is added
        ql.Settings.instance().evaluationDate = vars.today

        missing_fixing_date = ql.Date(2, ql.January, 2023) # Assume this is not added
        self.assertTrue(vars.calendar.isBusinessDay(missing_fixing_date)) # Ensure it's a valid date

        with self.assertRaisesRegex(RuntimeError, "Missing .*eqIndex.* fixing for January 2nd, 2023"):
            vars.equity_index.fixing(missing_fixing_date)

    def test_error_when_interest_handle_missing(self):
        print("Testing error when interest handle is missing...")
        vars = CommonVarsEquityIndex()
        ql.Settings.instance().evaluationDate = vars.today
        forecasted_date = ql.Date(20, ql.May, 2030)

        # Clone with empty interest handle
        equity_index_no_ir = vars.equity_index.clone(
            ql.YieldTermStructureHandle(), # Empty interest handle
            ql.YieldTermStructureHandle(), # Empty dividend handle (original test had this too)
            ql.QuoteHandle()               # Empty spot handle (original test had this too)
        )
        # Re-add today's fixing if needed for forecasting from past
        if vars.equity_index.hasHistoricalFixings(): # Check if original had it
             equity_index_no_ir.addFixing(vars.today, 8690.0)


        with self.assertRaisesRegex(RuntimeError, "null interest rate term structure"):
            equity_index_no_ir.fixing(forecasted_date)

    def test_fixing_observability(self):
        print("Testing observability of index fixings...")
        vars = CommonVarsEquityIndex()
        ql.Settings.instance().evaluationDate = vars.today

        # Using a Python-native observer for simplicity
        class SimpleFlagObserver:
            def __init__(self):
                self.updated = False
            def update(self):
                self.updated = True
            def lower(self):
                self.updated = False
            def isUp(self):
                return self.updated

        flag = SimpleFlagObserver()

        # Create an index instance to observe
        # For fixings to be shared, they must have the same name and be managed by IndexManager
        index_name_obs = "observableEquityIndex"
        i1 = ql.EquityIndex(index_name_obs, vars.calendar, ql.EURCurrency())
        i1.registerObserver(flag) # QL objects are Observables, register Python observer directly

        flag.lower() # Reset flag

        # Create another instance with the same name and add a fixing
        # This relies on IndexManager propagating the fixing.
        i2 = ql.EquityIndex(index_name_obs, vars.calendar, ql.EURCurrency())
        i2.addFixing(vars.today, 100.0)

        self.assertTrue(flag.isUp(), "Observer was not notified of added equity index fixing")

    def test_no_error_if_today_is_not_business_day(self):
        print("Testing that no error is thrown if today is not a business day for forecast...")
        vars = CommonVarsEquityIndex()

        non_biz_today = ql.Date(28, ql.January, 2023) # Saturday
        self.assertFalse(vars.calendar.isBusinessDay(non_biz_today))
        ql.Settings.instance().evaluationDate = non_biz_today # Set eval date to non-biz day

        forecasted_date = ql.Date(20, ql.May, 2030)

        # Clone equity index. The original vars.equity_index might have a fixing for its vars.today.
        # We need a clean index or one whose fixings don't interfere with this specific eval date.
        # The clone operation is important here.
        # C++: vars.equityIndex->clone(vars.interestHandle, vars.dividendHandle, Handle<Quote>());
        # This clone has no spot handle. It will try to forecast from latest historical fixing.
        # If `add_todays_fixing` in CommonVars was true, `vars.today` (Jan 27) has a fixing.
        # When eval date is Jan 28 (non-biz), it should still use the latest available fixing (Jan 27).

        # Let's ensure the index used for this test has the necessary historical fixings.
        # vars.equity_index was created with original vars.today.
        # If we want to forecast FROM non_biz_today, the index needs its state set up correctly.
        # The point of the test is that `equityIndex->fixing(forecastedDate)` should work
        # even if `Settings::instance().evaluationDate()` is a non-business day.
        # It should still use the provided curves, which are typically defined for business days.

        # The clone in C++ uses Handle<Quote>() -> empty spot handle.
        # It forecasts from the latest available fixing.
        equity_index_test = vars.equity_index.clone(
            vars.interest_handle, vars.dividend_handle, ql.QuoteHandle() # No spot
        )
        # Ensure the original fixing from CommonVars is on the cloned index for forecasting from past
        equity_index_test.addFixing(vars.today, 8690.0) # Original vars.today (Jan 27)

        try:
            forecast_val = equity_index_test.fixing(forecasted_date)
            # Check it's a valid number
            self.assertTrue(not math.isnan(forecast_val) and not math.isinf(forecast_val),
                            f"Forecast value is invalid: {forecast_val}")
        except Exception as e:
            self.fail(f"EquityIndex.fixing threw an unexpected error when today is non-business day: {e}")


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)